In [4]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import os
from jupyterthemes import jtplot
from datetime import datetime
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
jtplot.style(theme='solarizedd')
plt.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

ROOT_DIR = '../..'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
DATA_RAW = os.path.join(DATA_DIR, 'raw')
DATA_INTERIM = os.path.join(DATA_DIR, 'interim')
DATA_EXTERNAL = os.path.join(DATA_DIR, 'external')

SRC_DIR = os.path.join(ROOT_DIR, 'src')

BOSTON_CALENDAR = os.path.join(DATA_RAW, 'boston', 'calendar.csv')
BOSTON_LISTINGS = os.path.join(DATA_RAW, 'boston', 'listings.csv')
BOSTON_REVIEWS = os.path.join(DATA_RAW, 'boston', 'reviews.csv')

BOSTON_LISTINGS_COLS = os.path.join(
    DATA_INTERIM, 'boston', 'listings_cols_df.pkl')

import sys
sys.path.append(SRC_DIR)
sys.path.append(os.path.join(SRC_DIR, 'data'))

import preprocessing as pp
import missing_data as md
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
calendar, listings, reviews = pp.load_data(city='boston')
calendar, listings, reviews = pp.transform_all(calendar,
                                               listings,
                                               reviews,
                                               save_results=True)
calendar, listings, reviews = md.fill_missing(calendar,
                                              listings,
                                              reviews,
                                              save_results=True)

/Users/antelinvestigacionydesarrollo/anaconda3/envs/data/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:907: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)


In [5]:
utils.show_data(calendar)

The data has shape: (1308890, 4)

There is 0 missing data!

   listing_id       date  available  price
0    12147973 2017-09-05      False  250.0
1    12147973 2017-09-04      False  250.0
2    12147973 2017-09-03      False  250.0
3    12147973 2017-09-02      False  250.0
4    12147973 2017-09-01      False  250.0


In [6]:
utils.show_data(listings)

The data has shape: (3585, 101)

There is 15006 missing data!

         id                            listing_url       scrape_id  \
0  12147973  https://www.airbnb.com/rooms/12147973  20160906204935   
1   3075044   https://www.airbnb.com/rooms/3075044  20160906204935   
2      6976      https://www.airbnb.com/rooms/6976  20160906204935   
3   1436513   https://www.airbnb.com/rooms/1436513  20160906204935   
4   7651065   https://www.airbnb.com/rooms/7651065  20160906204935   

  last_scraped                                           name  \
0   2016-09-07                     Sunny Bungalow in the City   
1   2016-09-07              Charming room in pet friendly apt   
2   2016-09-07               Mexican Folk Art Haven in Boston   
3   2016-09-07  Spacious Sunny Bedroom Suite in Historic Home   
4   2016-09-07                            Come Home to Boston   

                                             summary  \
0  Cozy, sunny, family home.  Master bedroom high...   
1  Charming a

In [9]:
listings.isnull().mean().sort_values(ascending=False).head(15)

has_availability                1.000000
neighbourhood_group_cleansed    1.000000
jurisdiction_names              1.000000
interaction                     0.433473
access                          0.415342
house_rules                     0.332497
market                          0.003905
city                            0.000558
host_is_superhost               0.000000
host_thumbnail_url              0.000000
host_picture_url                0.000000
host_neighbourhood              0.000000
host_listings_count             0.000000
host_total_listings_count       0.000000
host_verifications              0.000000
dtype: float64

Not so bad. market and city can be easily filled. The features that are not in the Seattle dataset can be dropped (to be able to compare). neighbourhood_group_cleansed may be filled with the neighbour values.